In [ ]:
import glob
import os
from sklearn.neighbors import NearestNeighbors
from PIL import Image, ImageDraw
import scanpy as sc
from core_functions.unrolling import *

##### Put in the paths to to the human adatas. Images will be exported for which the villi should be labeled in labelme

In [ ]:
data_dir = "/mnt/sata1/Analysis_Alex/human_r1"
input_folders = glob.glob(os.path.join(data_dir, "human_0*"))

In [ ]:
def create_base_image_type(adata, downsize=20):

    points = adata.obsm["X_spatial"] / downsize
    types = adata.obs["topic"].values

    # Define the size of the image (adjust as needed)
    image_width = 2000
    image_height = 2000

    # Create a white canvas as the base image
    base_image = Image.new("RGB", (image_width, image_height), (255, 255, 255))

    # Draw the points on the image
    draw = ImageDraw.Draw(base_image)
    point_size = 1  # Size of the points
    ct = 0
    for point in points:
        draw.ellipse(
            (
                point[0] - point_size,
                point[1] - point_size,
                point[0] + point_size,
                point[1] + point_size,
            ),
            fill=adata.uns["topic_colors"][
                np.where(adata.obs["topic"].cat.categories == types[ct])[0]
            ][0],
            outline=adata.uns["topic_colors"][
                np.where(adata.obs["topic"].cat.categories == types[ct])[0]
            ][0],
            width=0,
        )
        ct += 1

    return base_image

In [ ]:
for input_file in input_folders:
    try:
        adata = sc.read(
            os.path.join(input_file, "adatas", "03_intial_neighborhoods.h5ad")
        )
        all_spatial = adata.obsm["X_spatial"]

        try:
            adata.X = adata.X.A
        except:
            print("Adata already in array format")

        downsize = 4
        base_image = create_base_image_type(adata, downsize=downsize)
        plt.imshow(base_image)
        plt.show()
        file_path = os.path.join(input_file, "label_img.png")
        base_image.save(file_path)
        adata.uns["unrolling_downsize"] = downsize
        adata.write(os.path.join(input_file, "adatas", "03_intial_neighborhoods.h5ad"))
    except:
        print(input_file + " not generated yet")

### Label the villi in all images in labelme. We store our labels in the ./labels/filtering folder in this directory